# Build an Agent

By themselves language models can't take actions - they just output text. A big case for LangChain is creating agents. Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed or it is okay to finish. 

In this tutorial we will build an agent that can interact with multiple tools: one being a local database, the other being a search engine. You will be able to ask this agent questions, watch it call tools, and have a conversation with it.

## Concepts

- Using [language models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), in particular their ability to call tools,
- Creating a [Retriever](https://python.langchain.com/v0.2/docs/concepts/#retrievers) to expose specific information to our agent,
- Using a [Search Tool](https://python.langchain.com/v0.2/docs/concepts/#tools) to look up things online,
- Using [LangGraph Agents](https://python.langchain.com/v0.2/docs/concepts/#agents) which use an LLM to think about what to do and then execute upon that,
- Debugging and tracing your application using [LangSmith](https://python.langchain.com/v0.2/docs/concepts/#langsmith)
  - *We will probably not be doing this as we're not paying for LangSmith access*
  - I think this is possible with *LangServe Playground* if we really need it.

## Setup

We're expected to be using a jupyter notebook for this (and it seems like all) tutorials. 

## Define Tools

We first need to create the tools we want to use. We will use two tools today: [Tavily](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/) (to search online) and then a retriever over a local index that we will create.

### [Tavily](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/)

**OH LOOK ANOTHER PAYWALL IN THE TUTORIALS. THIS IS GETTING OLD!** It appears as though we also have the ability to use duckduckgo as a search engine. It's free, so we will be using that instead. We might have to find a better way to do web scraping, but for now this will suffice.

```ps
pip install -U duckduckgo-search
```

In [ ]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("what is the weather in SF")

### Retriever

We will also create a retriever over some data of our own. For a deeper explanation of each step here see [this tutorial](https://python.langchain.com/v0.2/docs/tutorials/rag/).

In order to run the following script on windows we need to install faiss. It is only available as a CPU version on windows.

```ps
pip install faiss-cpu
```

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OllamaEmbeddings(model="mistral:v0.3"))
retriever = vector.as_retriever()


In [ ]:
retriever.invoke("How to upload a dataset?")[0]

Now that we have populated our index that we will do retrieval over we can easily turn it into a tool (the format needed for an agent to properly use it).

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Tools

Now that we have created both we can create a list of tools that we will use downstream.

In [ ]:
toolList = [retriever_tool, search]

### Using Language Models

Next let's learn how to use a language model to call tools. LangChain supports many language models that you can use interchangeably - select the one that you want to use below (we'll be using a local model).

As of yet they don't support passing tools to a local model. We have to run two separate instances of the model to accomplish what should only take one. This is definitely an area of future improvement.

```ps
pip install langchain_experimental
```

In [ ]:
from langchain_community.chat_models import ChatOllama
chatModel = ChatOllama(model="mistral:v0.3", base_url="http://localhost:11434")
from langchain_experimental.llms.ollama_functions import OllamaFunctions
functionModel = OllamaFunctions(model="mistral:v0.3", base_url="http://localhost:11434")

You can call the language model by passing in a list of messages. The response is a `content` string by default.

In [ ]:
from langchain_core.messages import HumanMessage

response = chatModel.invoke([HumanMessage(content="Hi!")])
response.content

We can now see what it is like to enable this model to do tool caling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools.

In [ ]:
model_with_tools = functionModel.bind_tools(toolList)

We can now call the model. Let's first call it with a normal message and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

It seems like the tools are in the wrong format. I'll need to figure out how to properly format them to work with `ollama_functions`; both `chatollama` and `ollama_functions` aren't compatible with the way the tools are defined here. We'll have to find custom tooling to get this to work.

[ollama_functions docs](https://api.python.langchain.com/en/latest/llms/langchain_experimental.llms.ollama_functions.OllamaFunctions.html)  
[ollama_functions docs2](https://python.langchain.com/v0.1/docs/integrations/chat/ollama_functions/)  
[BaseTool docs](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool)  

After doing some further digging it seems like the ollama api and the way langchain calls it aren't compatible with tool use. If I want to do this I'll have to go lower and start with the `ollama` api directly. This is a bit of a bummer, but it's not the end of the world. I'll have to do some more digging to figure out how to get this to work.
